In [9]:
import os
import sys
import pickle
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split




In [10]:
with open("../../pickles/X_train_aug.pkl", "rb") as file:
    X_train = pickle.load(file)

with open("../../pickles/y_train_aug.pkl", "rb") as file:
    y_train = pickle.load(file)

with open("../../pickles/labels.pkl", "rb") as file:
    labels = pickle.load(file)

# with open("../../pickles/resized_images.pkl", "rb") as file:
#     resized = pickle.load(file)




KeyboardInterrupt: 

In [ ]:
print(labels)
print(labels.count) 

In [ ]:

print(X_train[0])
print(y_train[0])

print(X_train.shape)
print(y_train.shape)    

print(X_train.shape[1:]) 



In [ ]:
print(np.unique(y_train))  # Ensure this matches expected labels (0 to 5)
count = np.unique(y_train, return_counts=True)
print(count) 

In [6]:
y_encoder = LabelEncoder().fit(y_train)
y = y_encoder.transform(y_train)

# Convert values to numpy arrays
X = np.array(X_train)

# Split the training dataset into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)



In [ ]:
import keras_tuner as kt
from tensorflow import keras

def build_cnn_model(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(
            filters=hp.Int('filters', min_value=32, max_value=128, step=32),
            kernel_size=hp.Choice('kernel_size', values=[3, 5]),
            activation='relu',
            input_shape=(150, 150, 3)  # Modify for your dataset
        ),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int('units', min_value=32, max_value=128, step=32),
            activation='relu'
        ),
        keras.layers.Dense(10, activation='softmax')  # Modify based on num_classes
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Initialize the RandomSearch Tuner
tuner = kt.RandomSearch(
    hypermodel=build_cnn_model,
    objective='val_accuracy',
    max_trials=2,  # Number of hyperparameter sets to try
    executions_per_trial=1,  # Number of times to train each model
    directory='hyperparameter_tuning',
    project_name='cnn_randomsearch'
)

# Load and preprocess dataset (Modify as needed)
(x_train, y_train), (x_val, y_val) = keras.datasets.mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0  # Normalize pixel values
x_train, x_val = x_train[..., None], x_val[..., None]  # Add channel dimension

# Start hyperparameter tuning
tuner.search(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=32)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters:", best_hps.values)

# Retrieve and build the best model
model = tuner.hypermodel.build(best_hps)
model.summary()

In [ ]:


# model = keras.Sequential([
#     layers.Input(shape=(150, 150, 3)),
#     layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
#     layers.MaxPooling2D(2,2),
#     layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
#     layers.MaxPooling2D(2,2),
#     layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
#     layers.MaxPooling2D(2,2),
#     layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
#     layers.MaxPooling2D(2,2),
#     layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
#     layers.MaxPooling2D(2,2),
#     layers.Flatten(),
#     layers.Dense(1024, activation='relu'),
#     layers.Dropout(0.2),
#     layers.Dense(128, activation='relu'),
#     layers.Dropout(0.2),
#     layers.Dense(6, activation='softmax')
# ]) 


# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs
)

In [ ]:
model.summary()

In [ ]:
model.save("../../models/keras_model.h5")

In [ ]:
img_path = "../../model/pred/93.jpg"  # Change this to your image path
img_path = "../../resources/buildings/66.jpg"  # Change this to your image path

img = image.load_img(img_path, target_size=(150, 150))  # Resize to match model input size
img.show()
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # Normalize if the model was trained with normalized images

# Expand dimensions to match the batch shape (model expects a batch, not a single image)
img_array = np.expand_dims(img_array, axis=0)
print(img_array.shape)
print(img_array.dtype)
# Make prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)
# Print predictions
text = predictions[0][predicted_class]
print(predictions)
print(predicted_class)

In [ ]:
predicted_class = np.argmax(predictions, axis=1)  # Get class index
print("Predicted Class:", predicted_class)